In [4]:
import sys
import pandas as pd
# Lägg till 'src' i sökvägen så att vi kan importera vår egen modul
sys.path.append('./src') 
from health_analyzer import HealthDataAnalyzer
from IPython.display import Markdown as md, display

In [5]:
analyzer = HealthDataAnalyzer('data/health_study_dataset.csv')

In [7]:
# Data Cleaning
display(md("## 1. Data Rengöring och Förberedelse"))
display(md("Enligt god praxis valideras datan även om inga problem förväntas."))

rengoring_resultat = analyzer.data_cleaning()
print(rengoring_resultat)

## 1. Data Rengöring och Förberedelse

Enligt god praxis valideras datan även om inga problem förväntas.

Rengöringsstatus:
- Saknade värden hittades: 0
- Dubbla rader hittades: 0
- Datan är nu redo för analys.
